## Load Libraries and Packages

In [73]:
import datetime
import numpy as np
from numpy import double
import pandas as pd

In [74]:
cd "/Users/2015/Dropbox/Data Science/NYC Data Science Academy/class-projects/Rossmann/KaggleProject/KaggleProject"

/Users/2015/Dropbox/Data Science/NYC Data Science Academy/class-projects/Rossmann/KaggleProject/KaggleProject


## Load Data

In [75]:
print("Loading data start...")
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
store = pd.read_csv("input/store.csv")
states = pd.read_csv("input/store_states.csv")
print("Complete!   : )")

Loading data start...
Complete!   : )


In [76]:
train.loc[(train['Open'] == 1) & ((train['Sales'] == 0) | (train['Customers'] == 0))].shape

(54, 9)

## Process Data:
#### 1: Impute open = 1 for missing open test dasta
#### 2: Separate date column into year, month, day
#### 3: Convert date column to type 'date' and extract:
* day_of_year
* week_of_year
* quarter
* month start
* month end
* quarter start
* quarter end

#### 4: Drop date and standardize StateHoliday
#### 5: Create promotion one feature
#### 6: Merge data
#### 7: Dumify categorical variables

In [77]:
# 1: Impute Open = 1 for store 622 in test data
test['Open'].fillna(1, inplace=True)
print 'Step 1 Complete.'

Step 1 Complete.


In [78]:
# 2: Separate date columns
train['year'] = train.Date.apply(lambda x: x.split('-')[0])
train['year'] = train['year'].astype(int)
train['month'] = train.Date.apply(lambda x: x.split('-')[1])
train['month'] = train['month'].astype(int)
train['day'] = train.Date.apply(lambda x: x.split('-')[2])
train['day'] = train['day'].astype(int)
print 'Step 2 Complete.'

Step 2 Complete.


In [79]:
# 3: Withdraw multiple date features
train['Date'] = pd.to_datetime(train['Date'])
train['day_of_year'] = train['Date'].dt.dayofyear
train['week_of_year'] = train['Date'].dt.weekofyear
train['quarter'] = train['Date'].dt.quarter
train['is_month_start'] = train['Date'].dt.is_month_start.astype(int)
train['is_month_end'] = train['Date'].dt.is_month_end.astype(int)
train['is_quarter_start'] = train['Date'].dt.is_quarter_start.astype(int)
train['is_quarter_end'] = train['Date'].dt.is_quarter_end.astype(int)
print 'Step 3 Complete.'

Step 3 Complete.


In [80]:
# 4: Drop date and standardize StateHoliday column.
# train = train.drop('Date',1)
train.ix[(train['StateHoliday'] == 0), 'StateHoliday'] = '0'
#train['StateHoliday'] = map(lambda x: '0' if x == 0 else x, train['StateHoliday'])
print 'Step 4 Complete.'

Step 4 Complete.


In [81]:
# 5: Create duration of promo 1
train = train.sort(columns = ['Store', 'Date'], ascending = True, na_position = 'last')
train['PromoFirstDate'] = 0
train.ix[(train['Promo'] == 1 ) & (train['Promo'].shift(1) ==0), 'PromoFirstDate'] = 1
print 'Step 5 Complete.'

Step 5 Complete.


/Users/2015/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [82]:
# 6: Merge store and location features with training data
tm_temp = pd.merge(train, store, on = 'Store', how = 'outer')
tm = pd.merge(tm_temp, states, on = 'Store', how = 'outer')
print 'Step 6 Complete.'


Step 6 Complete.


# Output CSV files for R

In [83]:
tm.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,1,2,2013-01-01,0,0,0,0,a,1,2013,...,c,a,1270,9,2008,0,NaN,NaN,NaN,HE
1,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,c,a,1270,9,2008,0,NaN,NaN,NaN,HE
2,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,c,a,1270,9,2008,0,NaN,NaN,NaN,HE
3,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,c,a,1270,9,2008,0,NaN,NaN,NaN,HE
4,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,c,a,1270,9,2008,0,NaN,NaN,NaN,HE


In [84]:
tm.to_csv("data/train_store_states_R.csv")

In [85]:
# 7: Create features for all state holidays, store types and assortments
tm_SH = pd.get_dummies(tm.StateHoliday,'StateHoliday')
tm_ST = pd.get_dummies(tm.StoreType, 'StoreType')
tm_AS = pd.get_dummies(tm.Assortment, 'Assortment')
tm_dummy = pd.concat([tm, tm_SH[tm_SH.columns[1:]]],axis = 1)
tm_dummy = pd.concat([tm_dummy, tm_ST[tm_ST.columns[1:]]],axis = 1)
tm_final = pd.concat([tm_dummy, tm_AS[tm_AS.columns[1:]]],axis = 1)
tm_final.drop(['StateHoliday', 'StoreType', 'Assortment'],1,inplace=True)
print 'Step 7 Complete.'

tm_final.head()

Step 7 Complete.


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,year,month,...,PromoInterval,State,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c
0,1,2,2013-01-01,0,0,0,0,1,2013,1,...,NaN,HE,1,0,0,0,1,0,0,0
1,1,3,2013-01-02,5530,668,1,0,1,2013,1,...,NaN,HE,0,0,0,0,1,0,0,0
2,1,4,2013-01-03,4327,578,1,0,1,2013,1,...,NaN,HE,0,0,0,0,1,0,0,0
3,1,5,2013-01-04,4486,619,1,0,1,2013,1,...,NaN,HE,0,0,0,0,1,0,0,0
4,1,6,2013-01-05,4997,635,1,0,1,2013,1,...,NaN,HE,0,0,0,0,1,0,0,0


In [86]:
tm_final.to_csv("data/train_store_states_Python.csv")